In [2]:
from datetime import datetime, date
import calendar
import pandas as pd
import os

os.chdir('/Users/edlyness/python_work/Lesson_organiser/files')

In [3]:
# this cell initialises the pupil dataframe with pupil_data.csv and normal lesson fees
lesson_length_fees = {20: 18.0, 30: 25.0, 60: 45.0, 90: 70.0} # {lesson_length : corresponding fee in £}

pupil_dataframe = pd.read_csv("/Users/edlyness/python_work/Lesson_organiser/files/pupil_data.csv")
pupil_dataframe['sibling?'] = pupil_dataframe['sibling?'].map(lambda x: False if x == 'False' else x) # makes sure those False values are boolean

lesson_dataframe = pd.read_csv("/Users/edlyness/python_work/Lesson_organiser/files/lesson_data.csv")
lesson_dataframe['date'] = lesson_dataframe['date'].map(lambda x: datetime.strptime(x, "%Y-%m-%d").date())

pupils = pupil_dataframe['name'].tolist()

#print(pupil_dataframe)
#print(lesson_dataframe)


In [11]:
# functions used to input data about a specific lesson

def get_date(prompt=""):
    valid = False

    print(prompt)
    
    while not valid:
        try:
            user_input = input()
            if user_input == "":
                return datetime.now().date()
            if user_input == "cancel": # the user can cancel any time, in which case it returns False
                return False
            
            else:
                date_parts = user_input.split('/') # splits the user input eg. '12/3/23' -> (12, 3, 23)
                if len(date_parts) > 3: # eg '12/3/4/32'
                    print("Input valid date")
                    continue
                day, month = map(int, date_parts[:2])
                if len(date_parts) == 2: # if input omitted year
                    year = datetime.now().year
                else:
                    year = int(date_parts[2])
                    if year < 100: # this turns eg '23' into '2023' for consistency
                        year += 2000
                return date(year, month, day)
                valid = True
        except:
            print("Input valid date")

def get_pupil(prompt=""): #NB this will return pupil code, not pupil name
    valid = False

    while not valid:
        user_input = input(prompt)
        
        if user_input == "cancel": # the user can cancel any time, in which case it returns False
            return False
        
        matching_pupils = [pupil for pupil in pupils if pupil.startswith(user_input.lower())] # makes a list of pupils whose name begins with user input. This allows for inputing eg 'ha' instead of 'harry'
        
        if matching_pupils: # checks if there are any matching pupils
            if len(matching_pupils) == 1: # will only accept there being one matching pupil to avoid ambiguity
                valid = True
                # next line returns the value stored in the code column, for the matching name stored in the name column. NB there can be no duplicate names using this method
                return pupil_dataframe.loc[pupil_dataframe['name'] == matching_pupils[0], 'code'].item()
            else:
                print("Multiple pupils found. Did you mean:") # shows user if there are more than one matching pupils.
                print(", ".join(matching_pupils))

        else:
            print(user_input + " is not a valid pupil")

def get_length(pupil_code): # returns lesson length
    #pupil_code is always a string (eg. 'p0') if it is only one pupil, or a tuple (eg. ('p7', 'p8')) if it is a double lesson
    
    valid = False

    while not valid:
        try:
            if isinstance(pupil_code, str): # if user was only teaching one pupil, not their sibling too
                user_input = input(str(pupil_dataframe.loc[pupil_dataframe['code'] == pupil_code, 'length'].item()) + " minute lesson?") # suggests usual lesson length

                if user_input == "cancel": # the user can cancel any time, in which case it returns False
                    return False

                elif user_input == "":
                    return pupil_dataframe.loc[pupil_dataframe['code'] == pupil_code, 'length'].item()
            else: # if it was a 'double lesson'
               
                user_input = input(str(pupil_dataframe.loc[pupil_dataframe['code'] == pupil_code[0], 'length'].item() + pupil_dataframe.loc[pupil_dataframe['code'] == pupil_code[1], 'length'].item()) + " minute lesson?") # suggests usual lesson length

                if user_input == "cancel": # the user can cancel any time, in which case it returns False
                    return False

                elif user_input == "":
                    return pupil_dataframe.loc[pupil_dataframe['code'] == pupil_code[0], 'length'].item() + pupil_dataframe.loc[pupil_dataframe['code'] == pupil_code[1], 'length'].item()

            # if user inputs a number rather than just hitting return, it accepts that as the lesson length
            length = int(user_input)

            if 10 <= length <= 120: # min and max values for lesson length in minutes
                valid = True
                return user_input
            else:
                print("lesson length must be between 10mins and 2hrs")
        except:
            print(str(user_input) + " is not a valid lesson length")

def get_fee(lesson_length): # returns lesson fee
    valid = False

    while not valid:
        try:
            user_input = input(f"paid £{lesson_length_fees[lesson_length]:.2f}?") # suggests usual lesson fee
            
            if user_input == "cancel": # the user can cancel any time, in which case it returns False
                return False
        
            elif user_input == "":
                return lesson_length_fees[lesson_length]
            
            # if user inputs a number rather than just hitting return, it accepts that as the lesson fee
            length = int(user_input)
            
            if 10 <= length <= 90: # min and max values for lesson fee in GBP
                valid = True
                return user_input
            else:
                print("lesson fee must be between £10 and £90")
        except:
            print(str(user_input) + " is not a valid lesson fee")

def lesson_report(): # returns False if the user inputs cancel at any time

    confirmed = False
    is_sibling_lesson = False
    
    while not confirmed:
        #lesson_code = "L" + str((int(lesson_keys[-1][1:]) + 1))
        
        lesson_code = "L" + str(int(lesson_dataframe['code'].iloc[-1][1:]) + 1)

        lesson_pupil = get_pupil("Who were you teaching? ") # NB this returns the pupil code, or an empty string if the user inputs 'cancel'
        
        if not lesson_pupil: # ie if the user inputted 'cancel'
            return pd.DataFrame([])
        
        # if the pupil has a sibling
        elif pupil_dataframe.loc[pupil_dataframe['code'] == lesson_pupil, 'sibling?'].item(): 
            # print("Sibling = " + pupil_dataframe.loc[pupil_dataframe['code'] == lesson_pupil, 'sibling?'].item())

            sibling_code = pupil_dataframe.loc[pupil_dataframe['code'] == lesson_pupil, 'sibling?'].item() # gets the sibling's pupil code
            #print(pupil_dataframe.loc[pupil_dataframe['code'] == sibling_code, 'name'])
            user_input = input("Did you also teach " + pupil_dataframe.loc[pupil_dataframe['code'] == sibling_code, 'name'].item() + "? (y/n)").lower()
            if user_input == "y":
                is_sibling_lesson = True
                lesson_pupil = (lesson_pupil, sibling_code)
            elif user_input == "cancel":
                return pd.DataFrame([])

        lesson_date = get_date("When was this lesson")
        if not lesson_date: # ie if the user inputted 'cancel'
            return pd.DataFrame([])

        lesson_length = get_length(lesson_pupil)
        if not lesson_length: # ie if the user inputted 'cancel'
            return pd.DataFrame([])

        lesson_fee = get_fee(lesson_length)
        if not lesson_fee: # ie if the user inputted 'cancel'
            return pd.DataFrame([])
        
        new_row_dict = {
            'code' : lesson_code,
            'date' : lesson_date,
            'pupil_code' : lesson_pupil,
            'length' : lesson_length,
            'fee' : lesson_fee
        }

        print("\nHere are the lesson details for you to confirm:")
        if not is_sibling_lesson:
            print("pupil        : " + pupil_dataframe.loc[pupil_dataframe['code'] == lesson_pupil, 'name'].item().title())
        else:
            print("pupils       : " + pupil_dataframe.loc[pupil_dataframe['code'] == lesson_pupil[0], 'name'].item().title() + " and " + pupil_dataframe.loc[pupil_dataframe['code'] == lesson_pupil[1], 'name'].item().title())
         
        print("date         : " + lesson_date.strftime("%A %d/%m/%Y\n") +
              "lesson length: " + str(lesson_length) + " minutes\n" +
              f"fee          : £{lesson_fee:.2f}")

        user_input = input("Confirm? (y/n)")
        if user_input == "y" or user_input == "":
            new_row_df = pd.DataFrame([new_row_dict])
            return new_row_df
        elif user_input == "cancel":
            return pd.DataFrame([])



In [6]:
# other functions

def get_month_and_year(): # returns a tuple (month, year). Month value is 0 if user only inputs a year
    valid = False
    while not valid:
        user_input = input()
        if " " in user_input:
            date_parts = user_input.split(" ")
            if len(date_parts) == 2:
                #gets the month
                try:
                    if len(date_parts[0]) == 3:
                        month = datetime.strptime(date_parts[0], "%b").month
                    else:
                        month = datetime.strptime(date_parts[0], "%B").month
                except:
                    print("Input valid time")
                    continue
                try:
                    if int(date_parts[1]) < 100:
                        year = 2000 + int(date_parts[1])
                        valid = True
                    else:
                        year = int(date_parts[1])
                        valid = True
                except:
                    print("Input valid time")
                    continue
        else:
            month = 0
            try:
                if int(user_input) < 100:
                    year = 2000 + int(user_input)
                    valid = True
                else:
                    year = int(user_input)
                    valid = True
            except:
                print("Input valid time")
                continue

        return month, year

def print_last_ten_lessons():
    last_ten_lessons_df = lesson_dataframe.sort_values(by='date', ascending=False).head(10)
    
    user_friendly_ten_lessons_df = pd.DataFrame([])
    user_friendly_ten_lessons_df['Date'] = last_ten_lessons_df['date'].map(lambda x: x.strftime("%A %d/%m/%Y"))
    pupil_list = last_ten_lessons_df['pupil_code'].tolist()
    for index, pupil in enumerate(pupil_list):
        if isinstance(pupil, str):
            pupil_list[index] = pupil_dataframe.loc[pupil_dataframe['code'] == pupil, 'name'].item().title()
        else:
            pupil_list[index] = pupil_dataframe.loc[pupil_dataframe['code'] == pupil[0], 'name'].item().title() + " and " + pupil_dataframe.loc[pupil_dataframe['code'] == pupil[1], 'name'].item().title()
    user_friendly_ten_lessons_df['Pupil'] = pupil_list
    user_friendly_ten_lessons_df['Length'] = last_ten_lessons_df['length'].map(lambda x: str(x) + " mins")
    user_friendly_ten_lessons_df['Fee'] = last_ten_lessons_df['fee'].map(lambda x:f"£{x:.2f}")
    
    return user_friendly_ten_lessons_df.to_string(index=False)

In [ ]:
user_input = input("Hello Ed. Would you like to report a new lesson? (y/n) ").lower()

while True:
    if user_input == "y":
        new_lesson_df = lesson_report()
        lesson_dataframe = pd.concat([lesson_dataframe, new_lesson_df], ignore_index = True)
        print(lesson_dataframe)
                                          
        user_input = input("Report another lesson? (y/n)")
        if user_input != "y":
            save_user_input = input("Do you want to save? (y/n)")
            if user_input == "y":
                lesson_dataframe.to_csv("/Users/edlyness/python_work/Lesson_organiser/files/pupil_data.csv", index=False)
    else:
        print("Here are some options:\nLesson history (h)\nAdd new pupil (a)\nback (b)")
        user_input = input().lower()
        if user_input == "h":
            total_earned = 0
            total_hours = 0
            matching_lessons = []
            
            print_ten_most_recent_lessons()
            
            print("View lessons from a certain time (t) or from a certain pupil (p)?")
            user_input = input().lower()
            
            if user_input == "t":
                print("input time")
                month, year = get_month_and_year()
                if month == 0:
                    time_formatted = str(year)
                    for lesson_code, date in lesson_dates.items():
                        if date.year == year:
                            matching_lessons.append(lesson_code)
                    total_lessons = len(matching_lessons)
                else:
                    time_formatted = calendar.month_name[month] + " " + str(year)
                    for lesson_code, date in lesson_dates.items():
                        if date.year == year and date.month == month:
                            matching_lessons.append(lesson_code)
                    total_lessons = len(matching_lessons)

                for lesson_code in matching_lessons:
                    total_earned += lesson_fees[lesson_code]
                    total_hours += lesson_lengths[lesson_code]
                
                input(f"In {time_formatted} you earned £{total_earned:.2f}\npress enter to continue")
            elif user_input == "p":
                pupil = get_pupil("input pupil")
                for lesson_code, pupil_code in lesson_pupils.items():
                    if pupil == pupil_code:
                        matching_lessons.append(lesson_code)
                total_lessons = len(matching_lessons)
                
                for lesson_code in matching_lessons:
                    total_earned += lesson_fees[lesson_code]
                    total_hours += lesson_lengths[lesson_code]
                
                input(f"You have earned £{total_earned:.2f} teaching {pupil_codes[pupil].title()}.\npress enter to continue")
        
        

In [ ]:
lesson_dataframe



2023-06-05


## type(lesson_dataframe.loc[3, 'date'])

Program for helping keep track of piano lessons

For each lesson it should store a unique lesson key, the pupil, the date, the time and the fee
	Stored in a series of dictionaries - the key is the lesson code, the value is the name/date/length/fee etc.

	Each pupil should have a dictionary as well, with all their previous lesson keys stored in them.

Info to store about each pupil - unique pupil key, name, normal lesson length, normal lesson fee, as well as a record of all previous lessons']



On opening program:
1: Greet Ed, say would you like to report new lesson?
2: if yes, then report lesson, goto 1
3: else, 

Here are some options
View lessons (v)
Add new pupil (a)
